In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Georgetown,MY,2022-02-03 15:16:37,5.4112,100.3354,82.89,89,20,1.14
1,1,Chuy,UY,2022-02-03 15:18:29,-33.6971,-53.4616,87.12,51,24,4.92
2,2,Carutapera,BR,2022-02-03 15:18:29,-1.1950,-46.0200,82.40,80,99,7.29
3,3,New Norfolk,AU,2022-02-03 15:18:29,-42.7826,147.0587,51.71,64,3,1.99
4,4,Airai,TL,2022-02-03 15:18:30,-8.9266,125.4092,60.51,93,99,2.62


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Georgetown,MY,2022-02-03 15:16:37,5.4112,100.3354,82.89,89,20,1.14
1,1,Chuy,UY,2022-02-03 15:18:29,-33.6971,-53.4616,87.12,51,24,4.92
2,2,Carutapera,BR,2022-02-03 15:18:29,-1.1950,-46.0200,82.40,80,99,7.29
10,10,Butaritari,KI,2022-02-03 15:18:32,3.0707,172.7902,80.85,80,78,12.82
12,12,Atar,MR,2022-02-03 15:18:32,20.5169,-13.0499,87.69,6,0,13.27
13,13,Cape Town,ZA,2022-02-03 15:16:00,-33.9258,18.4232,75.11,55,20,4.00
14,14,Rikitea,PF,2022-02-03 15:16:00,-23.1203,-134.9692,77.36,77,100,13.31
18,18,Bambous Virieux,MU,2022-02-03 15:14:53,-20.3428,57.7575,80.85,89,75,25.32
19,19,Mar Del Plata,AR,2022-02-03 15:17:39,-38.0023,-57.5575,75.09,84,100,1.99
24,24,Calafell,ES,2022-02-03 15:18:35,41.2000,1.5683,75.34,70,93,7.00


In [8]:
preferred_cities_df.count()


City_ID       189
City          189
Country       187
Date          189
Lat           189
Lng           189
Max Temp      189
Humidity      189
Cloudiness    189
Wind Speed    189
dtype: int64

In [9]:
clean_city_df=preferred_cities_df.dropna()

In [10]:
clean_city_df.count()

City_ID       187
City          187
Country       187
Date          187
Lat           187
Lng           187
Max Temp      187
Humidity      187
Cloudiness    187
Wind Speed    187
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,82.89,5.4112,100.3354,
1,Chuy,UY,87.12,-33.6971,-53.4616,
2,Carutapera,BR,82.40,-1.1950,-46.0200,
10,Butaritari,KI,80.85,3.0707,172.7902,
12,Atar,MR,87.69,20.5169,-13.0499,
13,Cape Town,ZA,75.11,-33.9258,18.4232,
14,Rikitea,PF,77.36,-23.1203,-134.9692,
18,Bambous Virieux,MU,80.85,-20.3428,57.7575,
19,Mar Del Plata,AR,75.09,-38.0023,-57.5575,
24,Calafell,ES,75.34,41.2000,1.5683,


In [12]:
hotel_df.count()

City          187
Country       187
Max Temp      187
Lat           187
Lng           187
Hotel Name    187
dtype: int64

In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECptN2geDjwtphbDtIfFEP6zvLhekZxPvMh41uJjBG-N_DrsP2KmXh68RrpBntS1LSp5xI-Vkmw4loz9gEIkzbeSuDY2stvrajOVAE1X3jGNVv7eiiljie5O5XhgGYX5SUOc5coAQyxkuhWdnyX3Cf-ldf07lsHRFGzrnFNl8FQoOJmQr8OY6WHALplUkGwKdh0PHf18S9J_0llQ4gCaTAhx6fPNh8cF8XgdvxstOvwDixp27SQXVrGmWs20QUbRH9BIpIyyIODXrwkJjT3oIGFCgI3i_MK6OtMI4QaMgwTmF1mHrg5NzIsBAuw30aS6dZ7xMt-H3KvNClhp-_jugJPTqZTyqRxcF4KU0YYTueprWCkNTW4X10M7YweGfV5EOHPeiAf6bGvun5GsR4FBEe88yx501YH5vS-kkwQZMHVYaO0SAOnfBBH',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [15]:
len(hotels["results"])

20

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Georgetown,MY,82.89,5.4112,100.3354,Cititel Penang
1,Chuy,UY,87.12,-33.6971,-53.4616,Nuevo Hotel Plaza
2,Carutapera,BR,82.40,-1.1950,-46.0200,Lidera
10,Butaritari,KI,80.85,3.0707,172.7902,Isles Sunset Lodge
12,Atar,MR,87.69,20.5169,-13.0499,Oumou Elghoura
...,...,...,...,...,...,...
584,Innisfail,AU,77.13,-17.5333,146.0333,Moondarra Motel
595,Rio Verde De Mato Grosso,BR,84.72,-18.9181,-54.8442,Quedas Palace Hotel
596,Benjamin Constant,BR,77.13,-4.3831,-70.0311,CASA AVALIADA
598,Kampot,KH,75.25,10.6167,104.1833,Two Moons Hotel


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))